In [11]:
import requests  # b_soup_1.py
import re
from bs4 import BeautifulSoup
#spoof the user agent to make page scrapable
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
#define position search term (one word) and location
position='analyst'
location='Pittsburgh,+PA'
#in the future, will iterate through all pages with while loop
  #for now, just retrieve urls for first page
page_num=0
page_str=str(page_num)
#url for search query
url='https://www.indeed.com/jobs?q='+position+'&l='+location+'&start='+page_str
search_page=requests.get(url,headers=headers)
search_page

<Response [200]>

In [12]:
#page.content

In [13]:
soup=BeautifulSoup(search_page.content,'html.parser')
#print(soup.prettify())

In [14]:
#find all job posting cards
all_cards=soup.find_all(class_='jobsearch-SerpJobCard row result')
#display first job card
#all_cards[0]

In [15]:
#empty list to store job posting urls
job_urls=[]
#it seems to be easier to pull posting dates and locations from search page, so do that here
posting_dates=[]
locations=[]
#iterate through all_cards, extract job posting urls, posting dates, and locations
for card in all_cards:
    #urls
    job_id=card['data-jk']
    job_url='https://www.indeed.com/viewjob?jk='+job_id
    job_urls.append(job_url)
    #posting dates
    #some jobs don't have posting dates so try them
    try:
        job_post_date=card.find(class_='date').get_text()
    except:
        job_post_date=' '
    posting_dates.append(job_post_date)
    #locations
    job_location=card.find(class_='location').get_text()
    locations.append(job_location)
#locations

In [16]:
#lets make this dataframe
#declare the empty lists that will become columns
position_name=[]
employer=[]
full_description=[]
#in the future, we can develop a list of keywords that we'll 
  #search postings for, but for now, just return all capitalized
   #words besides common words
capital_words=[]


In [17]:
#iterate through urls, retrieve info from job posting
for job_url in job_urls:
    #retrieve job page
    job_page=requests.get(job_url,headers=headers)
    #make job page into BeautifulSoup object
    job_soup=BeautifulSoup(job_page.content,'html.parser')
    #retrieve the job title
    job_position=job_soup.find(class_='icl-u-xs-mb--xs').get_text()
    #retrieve company name from job rating line on page
    job_rating_line=job_soup.find(class_='jobsearch-InlineCompanyRating')
    job_employer=job_rating_line.find(class_='icl-u-lg-mr--sm').get_text()
    #retrieve full job description
    job_descrip=job_soup.find(class_='jobsearch-JobComponent-description').get_text()
    #replace line breaks in description with spaces
    job_descrip.replace('\n',' ')
    #find capitalized words in job description
    capitals=re.findall('([A-Z][a-z]+)', job_descrip)
    #replace commonly capitalized words
    for x in ['The','Our','A','As','On','Using','We', 'In', 'Some']:
        try:
            capitals[:]=(value for value in capitals if value!=x)
        except:
            capitals
    #make capitals into a set instead of a list
    capitals=set(capitals)
    capital_string=', '.join(capitals)
    #append all of the items created to the empty lists from last step
    position_name.append(job_position)
    employer.append(job_employer)
    full_description.append(job_descrip)
    capital_words.append(capitals)

In [18]:
print(len(position_name),len(employer),len(locations),len(posting_dates),len(job_urls),
     len(full_description),len(capital_words))

14 14 14 14 14 14 14


In [19]:
#store results in dictionary, then data frame
import pandas as pd
result_dict={'Job Title':position_name,
            'Employer':employer,
            'Location':locations,
            'Posting Date':posting_dates, 
            'Posting Url':job_urls,
            'Full Job Description':full_description,
            'Keywords':capital_words
            }
result_frame=pd.DataFrame(result_dict)
result_frame

,Job Title,Employer,Location,Posting Date,Posting Url,Full Job Description,Keywords
0,Business Process Analyst,Polyconcept North America,"New Kensington, PA 15068",,https://www.indeed.com/viewjob?jk=42ad913a9fce...,Customer Operations Business Process Analyst\n...,"{Business, Service, Lead, Work, Spirit, Facili..."
1,Business Analyst,"Liberty Tire Recycling, LLC","Pittsburgh, PA 15212",2 days ago,https://www.indeed.com/viewjob?jk=2b6b7499b165...,"$50,000 - $55,000 a yearCompany OverviewLibert...","{Salary, Strong, Personal, Education, Work, Re..."
2,Market Analyst,Carmeuse Lime and Stone,"Pittsburgh, PA",,https://www.indeed.com/viewjob?jk=b56e0d79ec34...,Company Description: Carmeuse Lime & Stone bas...,"{Visualizations, Strategic, Pittsburgh, Arc, R..."
3,IT Business Analyst,Pepon inc,"Pittsburgh, PA",4 hours ago,https://www.indeed.com/viewjob?jk=e3e1de7c3972...,ContractExperience: 10+ Years business analyst...,"{Strong, Business, Pittsburgh, Years, Job, Poi..."
4,Business Analyst,Concept Technology LLC,"Pittsburgh, PA",11 days ago,https://www.indeed.com/viewjob?jk=b29010d4ac11...,"$50,000 - $60,000 a yearHello Everyone,Our rap...","{Phone, Singh, Salary, Business, System, Pay, ..."
5,Business Analyst II,UPMC,"Pittsburgh, PA 15219 (Middle Hill area)",13 days ago,https://www.indeed.com/viewjob?jk=2e3f63e8335b...,Community HealthChoices (CHC) is Pennsylvania'...,"{Salary, Hope, Bachelors, Strong, Extensive, O..."
6,Eligibility Analyst,Zenith American,"Pittsburgh, PA 15222 (Strip District area)",23 hours ago,https://www.indeed.com/viewjob?jk=d7653382d3e3...,Title: Eligibility Analyst Department: Contrib...,"{Personal, Cross, Work, Shield, Administrator,..."
7,Business Development Analyst,UnitedHealth Group,"Pittsburgh, PA 15289 (Central Business Distric...",12 days ago,https://www.indeed.com/viewjob?jk=7b6a7da620fe...,"Great sales are the result of strong purpose, ...","{Individual, Insurance, Go, Careers, Here, Gre..."
8,Operations Analyst,PNC Financial Services Group,"Pittsburgh, PA 15289 (Central Business Distric...",1 day ago,https://www.indeed.com/viewjob?jk=91009db8062e...,"Position Overview\n\nAt PNC, our people are ou...","{Customer, Education, Pittsburgh, Ave, Positio..."
9,Analyst,American Eagle Outfitters,"Pittsburgh, PA 15205",30+ days ago,https://www.indeed.com/viewjob?jk=9084c287f2ec...,As a member of the Customer Insights & Strateg...,"{Enlist, Business, Suite, Strong, Outfitters, ..."


In [21]:
result_frame.to_excel("draft output.xlsx")